In [1]:
import pandas as pd
import matplotlib.pyplot as plt
#from scipy.stats import ttest_ind
import sys
#from scipy.stats import chi2_contingency
import numpy as np

In [2]:
dir_fig = 'investigate_trs/'

In [3]:
#########################################
# generate the big table
#file_name = 'combined_table.tsv'
#df = get_big_table(file_name)

# or read the big table
file_name = 'test.tsv'
df = pd.read_table(file_name, sep='\t', header=0, keep_default_na=False)
#########################################

/scratch/smmortazavi/job_33476588/ipykernel_579396/1782051063.py:8: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(file_name, sep='\t', header=0, keep_default_na=False)


In [4]:
file_brainspan = '/expanse/projects/sebat1/miladm/UCSD/resources/brainspan_rnaseq/process/gene_list_RPKM_10.csv'
dtype = {'gene_id': str, 'entrez_id': str}
df_br = pd.read_table(file_brainspan, sep=',', header=0, dtype=dtype)
br_flt_10 = df_br.loc[df_br.mean_rpkm>10].gene_symbol.tolist()
br_flt_20 = df_br.loc[df_br.mean_rpkm>20].gene_symbol.tolist()
br_flt_30 = df_br.loc[df_br.mean_rpkm>30].gene_symbol.tolist()
br_flt_40 = df_br.loc[df_br.mean_rpkm>40].gene_symbol.tolist()
br_flt_50 = df_br.loc[df_br.mean_rpkm>50].gene_symbol.tolist()
print(df_br.shape)
print(len(br_flt_10))
print(len(br_flt_20))
print(len(br_flt_30))
print(len(br_flt_40))
print(len(br_flt_50))

(5824, 32)
5824
3045
1983
1392
1078


In [5]:
### this is for inspection/investigation only. the main version is up before this.
### previously discovered variants
file_fdr = '/expanse/projects/sebat1/miladm/UCSD/resources/ASD_TADA_2022/fdr_pvals_tada.tsv'
df_fdr = pd.read_table(file_fdr, sep='\t', header=0, index_col=None)
print('df_fdr:')
display(df_fdr)
asd_genes = df_fdr.loc[df_fdr.FDR_TADA_ASD <= 0.05].gene_gencodeV33.tolist()
dd_genes = df_fdr.loc[df_fdr.FDR_TADA_DD <= 0.05].gene_gencodeV33.tolist()
ndd_genes = df_fdr.loc[df_fdr.FDR_TADA_NDD <= 0.05].gene_gencodeV33.tolist()
print(f'asd: {len(asd_genes)}')
#print(f'asd: {(asd_genes)}')
print(f'dd: {len(dd_genes)}')
print(f'ndd: {len(ndd_genes)}')

file_pli = '/expanse/projects/sebat1/miladm/UCSD/resources/gnomAD/v4/Constraint/gene_max_pli.tsv'
df_pli = pd.read_table(file_pli, sep='\t', header=0, index_col=None).set_index('gene')
print('df_pli:')
display(df_pli)

file_lf = '/expanse/projects/sebat1/miladm/UCSD/resources/gnomAD/v4/Constraint/gene_min_loeuf.tsv'
df_lf = pd.read_table(file_lf, sep='\t', header=0, index_col=None).set_index('gene')
print('df_lf:')
display(df_lf)

print('dnLOF and dnMIS SNV/indels')
file_dn_snv = '/expanse/projects/sebat1/miladm/UCSD/resources/Antaki_2022/Antaki2022_supplementary_tables_ST5.tsv'
df_dn_snv = pd.read_table(file_dn_snv, sep='\t', header=0, index_col=None, keep_default_na=False)
df_dn_snv['PLI'] = df_dn_snv.gene.map(df_pli['lof.pLI'])
df_dn_snv['LOEUF'] = df_dn_snv.gene.map(df_lf['lof.oe_ci.upper'])
df_dn_snv['FDR_ASD'] = df_dn_snv.gene.map(lambda x: 1 if x in asd_genes else 0)
df_dn_snv['FDR_DD'] = df_dn_snv.gene.map(lambda x: 1 if x in dd_genes else 0)
df_dn_snv['FDR_NDD'] = df_dn_snv.gene.map(lambda x: 1 if x in ndd_genes else 0)
df_dn_snv['DEV_BR_GENE_10'] = df_dn_snv.gene.map(lambda x: 1 if x in br_flt_10 else 0)
df_dn_snv = df_dn_snv.loc[(df_dn_snv.iid.str.contains('REACH'))]
display(df_dn_snv.shape)
print(f" fdr asd: {df_dn_snv['FDR_ASD'].sum()}")
print(f" fdr dd: {df_dn_snv['FDR_DD'].sum()}")
print(f" fdr ndd: {df_dn_snv['FDR_NDD'].sum()}")
print(f" fdr brain genes: {df_dn_snv['DEV_BR_GENE_10'].sum()}")
print('df_dn_snv:')
display(df_dn_snv)
this_df_dn = df_dn_snv[['loss_of_function', 'missense', 'gene', 'PLI', 'LOEUF', 'FDR_ASD', 'FDR_DD', 'FDR_NDD', 'DEV_BR_GENE_10']].copy()
this_df_dn['samples_all'] = df_dn_snv['iid']
# filter constraint genes
this_df_dn = this_df_dn.loc[((this_df_dn.PLI>=0.9) | (this_df_dn.LOEUF<=0.37) | 
                             (this_df_dn.FDR_ASD==1) | (this_df_dn.FDR_DD==1) | (this_df_dn.FDR_NDD==1))]
print('this_df_dn:')
display(this_df_dn.shape)
display(this_df_dn)

this_df_dnL = this_df_dn.loc[this_df_dn.loss_of_function==1]
this_df_dnM = this_df_dn.loc[this_df_dn.missense==1]
print('this_df_dnL:')
display(this_df_dnL.shape)
print('this_df_dnM:')
display(this_df_dnM.shape)

print('inhLOF SNV/indels')
file_inh_snv = '/expanse/projects/sebat1/miladm/UCSD/resources/Antaki_2022/Antaki2022_supplementary_tables_ST8.tsv'
df_inh_snv = pd.read_table(file_inh_snv, sep='\t', header=0, index_col=None, keep_default_na=False)
df_inh_snv['PLI'] = df_inh_snv.gene.map(df_pli['lof.pLI'])
df_inh_snv['LOEUF'] = df_inh_snv.gene.map(df_lf['lof.oe_ci.upper'])
df_inh_snv['FDR_ASD'] = df_inh_snv.gene.map(lambda x: 1 if x in asd_genes else 0)
df_inh_snv['FDR_DD'] = df_inh_snv.gene.map(lambda x: 1 if x in dd_genes else 0)
df_inh_snv['FDR_NDD'] = df_inh_snv.gene.map(lambda x: 1 if x in ndd_genes else 0)
df_inh_snv['DEV_BR_GENE_10'] = df_inh_snv.gene.map(lambda x: 1 if x in br_flt_10 else 0)
df_inh_snv = df_inh_snv.loc[(df_inh_snv.cases.str.contains('REACH')) | ((df_inh_snv.controls.str.contains('REACH')))]
display(df_inh_snv.shape)
print(f" fdr asd: {df_inh_snv['FDR_ASD'].sum()}")
print(f" fdr dd: {df_inh_snv['FDR_DD'].sum()}")
print(f" fdr ndd: {df_inh_snv['FDR_NDD'].sum()}")
print(f" fdr brain genes: {df_inh_snv['DEV_BR_GENE_10'].sum()}")
print('df_inh_snv:')
display(df_inh_snv)
this_df_inh = df_inh_snv[['gene', 'PLI', 'LOEUF', 'FDR_ASD', 'FDR_DD', 'FDR_NDD', 'DEV_BR_GENE_10']].copy()
this_df_inh['samples_all'] = df_inh_snv.apply(lambda row: ','.join([x for x in [row.cases, row.controls, 
                                                                         row.fathers, row.mothers] if x != '']), axis=1)
# filter constraint genes
this_df_inh = this_df_inh.loc[((this_df_inh.PLI>=0.9) | (this_df_inh.LOEUF<=0.37) | 
                             (this_df_inh.FDR_ASD==1) | (this_df_inh.FDR_DD==1) | (this_df_inh.FDR_NDD==1))]
print('this_df_inh:')
display(this_df_inh.shape)
display(this_df_inh)

print('Merged denovo and inheritted one row per sample')
#this_df_dn_inh = pd.concat([this_df_dn, this_df_inh], axis=0, ignore_index=True)
this_df_dn_inh = pd.concat([this_df_dnL, this_df_inh], axis=0, ignore_index=True)
this_df_dn_inh['samples_all'] = this_df_dn_inh.samples_all.str.split(',')
this_df_dn_inh = this_df_dn_inh.explode('samples_all')

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
this_df_dn_inh['samples_all'] = this_df_dn_inh.apply(lambda row: rename_samples_dict[row['samples_all']] 
                                             if row['samples_all'] in rename_samples_dict else row['samples_all'], axis=1)

print('this_df_dn_inh:')
display(this_df_dn_inh)
this_df_dn_inh.to_csv('investigate.tsv', sep='\t', index=None)

df_fdr:


,gene_gencodeV33,FDR_TADA_ASD,FDR_TADA_DD,FDR_TADA_NDD,p_TADA_ASD,p_TADA_DD,p_TADA_NDD
0,SYNGAP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CHD8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ADNP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,SHANK3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,SCN2A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
18123,PSG2,0.911036,0.913971,0.879392,0.999770,0.999656,0.999762
18124,NaN,0.911038,0.913973,0.879395,0.999827,0.999713,0.999821
18125,GJB2,0.911040,0.913975,0.879399,0.999885,0.999771,0.999881
18126,PSG1,0.911042,0.913979,0.879403,0.999942,0.999885,0.999940


asd: 185
dd: 477
ndd: 664
df_pli:


,lof.pLI
gene,
A1BG,1.814400e-07
A1CF,6.691700e-03
A2M,7.412300e-02
A2ML1,1.996200e-01
A3GALT2,1.007500e-06
...,...
ZYG11A,3.084800e-08
ZYG11B,8.055900e-01
ZYX,3.807700e-02


df_lf:


...

this_df_inh:


(125, 8)

,gene,PLI,LOEUF,FDR_ASD,FDR_DD,FDR_NDD,DEV_BR_GENE_10,samples_all
275,MACF1,1.000000,0.126,0,0,0,1,"REACH000544,REACH000547,REACH000546"
281,PABPC4,1.000000,0.295,0,0,0,1,"REACH000760,REACH000759"
297,SLC2A1,1.000000,0.131,0,1,1,1,"REACH000650,REACH000652"
589,LMNA,1.000000,0.332,0,0,0,0,"REACH000372,REACH000430"
688,CACNA1E,1.000000,0.151,1,1,1,1,"REACH000668,REACH000670"
...,...,...,...,...,...,...,...,...
10597,SEC16A,0.000007,0.598,0,1,1,0,"REACH000668,REACH000669"
10615,TRAF2,1.000000,0.215,1,0,1,0,"REACH000166,REACH000168,REACH000164"
10745,EBP,0.936450,0.342,0,0,0,1,"REACH000586,REACH000587"
10774,AR,0.988370,0.291,0,0,0,0,"REACH000387,REACH000388"


Merged denovo and inheritted one row per sample
this_df_dn_inh:


,loss_of_function,missense,gene,PLI,LOEUF,FDR_ASD,FDR_DD,FDR_NDD,DEV_BR_GENE_10,samples_all
0,1.0,0.0,FBXO42,1.00000,0.271,0,0,0,0,REACH000718
1,1.0,0.0,R3HDM2,1.00000,0.366,0,0,0,1,REACH000097
2,1.0,0.0,PSMB5,0.99989,0.238,0,0,0,1,REACH000605
3,1.0,0.0,TSC2,1.00000,0.168,0,1,1,1,REACH000529
4,1.0,0.0,CTCF,1.00000,0.205,1,1,1,1,REACH000564
...,...,...,...,...,...,...,...,...,...,...
138,NaN,NaN,EBP,0.93645,0.342,0,0,0,1,REACH000587
139,NaN,NaN,AR,0.98837,0.291,0,0,0,0,REACH000387
139,NaN,NaN,AR,0.98837,0.291,0,0,0,0,REACH000388
140,NaN,NaN,CSTF2,0.99978,0.124,0,0,0,0,REACH000252


In [6]:
file_cov = '/expanse/projects/sebat1/s3/data/sebat/long_reads_pipeline_results/coverages_mod.tsv'
#file_cov = '/tscc/projects/ps-sebat1/s3/data/sebat/long_reads_pipeline_results/coverages_mod.tsv'
df_cov = pd.read_table(file_cov, header=0, sep='\t')
multiplat_samples = ['REACH000236', 'REACH000530', 'REACH000531', 'REACH000532', 'REACH000436']
df_cov['SAMPLE'] = df_cov.apply(lambda row: row['SAMPLE']+'_'+row['COHORT'] if row['SAMPLE'] in multiplat_samples else row['SAMPLE'], axis=1)
df_cov_high = df_cov.loc[df_cov.SAMPLE.str.startswith('REACH000') & (df_cov.MEAN_COVERAGE > 9)]
display(df_cov)
high_cov_samples = df_cov_high.SAMPLE.tolist()
#print(high_cov_samples)

def get_plat(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['COHORT'].values[0]
    except:
        print(f'problem with samples: {sample}')
        ret = ''
    return ret

plat_dict = {sample: get_plat(sample) for sample in df_cov.SAMPLE.tolist()}

,SAMPLE,COHORT,MEAN_COVERAGE
0,HG004,ONT,81.44
1,REACH000626,ONT,10.51
2,REACH000546,ONT,10.87
3,REACH000683,ONT,9.97
4,REACH000563,ONT,8.68
...,...,...,...
283,REACH000284,PB,3.43
284,REACH000672,PB,5.02
285,REACH000292,PB,9.00
286,REACH000066,PB,1.61


In [7]:
file_psam = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/process_IL_LR/HWE/REACH_LR_platform.psam'
df_p = pd.read_table(file_psam, sep='\t', header=None, names = ['famid', 'sample_id', 'dad', 'mom', 'sex', 'phen'])
display(df_p)

# make mom and dad dictionary
sample_dad_dict = {}
sample_mom_dict = {}
for sample, dad, mom in zip(df_p.sample_id, df_p.dad, df_p.mom):
    #print(sample, dad, mom)
    sample_dad_dict[sample] = dad
    sample_mom_dict[sample] = mom

print('complete trios:')
df_trios = df_p.loc[df_p.dad.isin(df_p.sample_id) & df_p.mom.isin(df_p.sample_id)]
display(df_trios)
print(df_trios.sex.value_counts())
print(df_trios.phen.value_counts())

parents_set_lr = set(df_trios.dad.tolist()) | set(df_trios.mom.tolist())
#print(f'len(parents_set_lr): {len(parents_set_lr)}')
#print(parents_set_lr)

parents_set_pb = set([x for x in parents_set_lr if plat_dict[x]=='PB'])
parents_set_ont = set([x for x in parents_set_lr if plat_dict[x]=='ONT'])
#print('len(parents_set_pb):', len(parents_set_pb))
#print('len(parents_set_ont):', len(parents_set_ont))

parent_kids_dict = {}
mom_kids_dict = {}
dad_kids_dict = {}
for parent in parents_set_lr:
    parent_kids_dict[parent] = df_trios.loc[(df_trios.dad == parent) | (df_trios.mom == parent)].sample_id.tolist()
    if parent in df_trios.dad.tolist():
        dad_kids_dict[parent] = df_trios.loc[(df_trios.dad == parent)].sample_id.tolist()
    if parent in df_trios.mom.tolist():
        mom_kids_dict[parent] = df_trios.loc[(df_trios.mom == parent)].sample_id.tolist()
print(f'len(parent_kids_dict): {len(parent_kids_dict)}')
#print(parent_kids_dict)
print(f'len(mom_kids_dict): {len(mom_kids_dict)}')
#print(mom_kids_dict)
print(f'len(dad_kids_dict): {len(dad_kids_dict)}')
#print(dad_kids_dict)

mate_dict = {}
for dad, mom in zip(df_trios.dad, df_trios.mom):
    mate_dict[dad] = mom
    mate_dict[mom] = dad
print(f'len(mate_dict): {len(mate_dict)}')
#print(mate_dict)

,famid,sample_id,dad,mom,sex,phen
0,2323,2323-2-1,0,0,2,2
1,2602,2602-2-1,0,0,2,2
2,3392,3392-2-3,0,0,2,2
3,3538,3538-2-1,0,0,2,2
4,3939,3939-3-1,0,0,2,2
...,...,...,...,...,...,...
275,F0270,REACH000681,REACH000683,REACH000682,1,2
276,F0270,REACH000682,0,0,2,2
277,F0270,REACH000683,0,0,1,2
278,F0270,REACH000684,REACH000683,REACH000682,1,2


complete trios:


,famid,sample_id,dad,mom,sex,phen
10,F0026,REACH000026,REACH000270,REACH000269,1,2
11,F0058,REACH000058,REACH000440,REACH000439,1,2
12,F0065,REACH000065,REACH000067,REACH000066,1,2
15,F0078,REACH000086,REACH000088,REACH000087,1,2
18,F0078,REACH000089,REACH000088,REACH000087,2,2
...,...,...,...,...,...,...
271,F0266,REACH000660,REACH000662,REACH000661,2,2
274,F0266,REACH000663,REACH000662,REACH000661,1,1
275,F0270,REACH000681,REACH000683,REACH000682,1,2
278,F0270,REACH000684,REACH000683,REACH000682,1,2


sex
1    74
2    43
Name: count, dtype: int64
phen
2    92
1    25
Name: count, dtype: int64
len(parent_kids_dict): 126
len(mom_kids_dict): 63
len(dad_kids_dict): 63
len(mate_dict): 126


In [8]:
meta_file = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/REACH_sample_info.tsv'
df_meta = pd.read_table(meta_file, sep='\t', header=0)

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_meta['Sample_ID'] = df_meta.apply(lambda row: rename_samples_dict[row['Sample_ID']] 
                                     if row['Sample_ID'] in rename_samples_dict else row['Sample_ID'], axis=1)
display(df_meta)

aff_dict = {}
for sample, aff in zip(df_meta['Sample_ID'].tolist(), df_meta['Affected'].tolist()):
    aff_dict[sample] = aff 

,Sample_ID,Subject_ID,Family_ID,DOB,Family_Type,Affected,Diagnosis_LAB,Mother_ID,Father_ID,Rel_Proband,...,Date sent for DNA extraction,Location of DNA extraction,DNA Box,Concentration (ng/uL),260/280,Date sent for Sequencing,WGS Provider,(Sample Well) Plate Name.Sample.ID,Affected_orig,Affected_ASD
0,REACH000001,F0001-01,F0001,3/26/2007,trio,Yes,Autism,F0001-02,F0001-03,Proband,...,Unknown,Gleeson,1,60.0,1.80,9/9/2013,Illumina,LP6005688-DNA_E11,Yes,Yes
1,REACH000107,F0001-02,F0001,8/9/1972,trio,Yes,Autism,Unknown,Unknown,Mom,...,Unknown,Gleeson,2,60.0,NaN,9/9/2013,Illumina,LP6005688-DNA_F11,Yes,Yes
2,REACH000226,F0001-03,F0001,9/27/1969,trio,Yes,"Autism, PTSD, ADHD",Unknown,Unknown,Dad,...,Unknown,Gleeson,3,60.0,NaN,9/9/2013,Illumina,LP6005688-DNA_G11,Yes,Yes
3,REACH000002,F0002-01,F0002,11/19/2007,incomplete,Yes,Autism,Unknown,Unknown,Proband,...,Unknown,Gleeson,1,Incomplete,Incomplete,Incomplete family,Incomplete family,Incomplete family,Yes,Yes
4,REACH000003,F0003-01,F0003,8/9/2000,incomplete,Yes,Autism,Unknown,Unknown,Proband,...,Unknown,Gleeson,1,Incomplete,Incomplete,Incomplete family,Incomplete family,Incomplete family,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,REACH000889,F0320-02,F0320,1/15/1981,multiple,No,"Crohn's Disease, Lupus (SLE)",F8888-88,F8888-88,Mom,...,2/28/2017,Sebat,11,36.2,1.79,pending,pending,pending,No,No
889,REACH000890,F0320-03,F0320,7/9/1980,multiple,No,NaN,F8888-88,F8888-88,Dad,...,2/28/2017,Sebat,11,41.9,1.84,pending,pending,pending,No,No
890,REACH000891,F0320-04,F0320,2/26/2010,multiple,No,NaN,F0320-02,F0320-03,Sibling,...,2/28/2017,Sebat,11,119.7,1.84,pending,pending,pending,No,No
891,REACH000892,F0320-05,F0320,10/14/2011,multiple,No,NaN,F0320-02,F0320-03,Sibling,...,2/28/2017,Sebat,12,134.1,1.85,pending,pending,pending,No,No


In [9]:
#file_pca = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/PCA/kid_pca_info.csv'
#file_pca = '/expanse/projects/sebat1/miladm/UCSD/LONG_READ_COHORT/PCA/SummaryGeneticData_REACH_SSC_SPARK.SBayesR.20210915.csv'
file_pca = '/expanse/projects/sebat1/j3guevar/SFARI/aspera-sf/SFARI_DS468588/data/master_phen_4.SBayesR.20210915.csv'

df_pca = pd.read_table(file_pca, sep=',', header=0)
cols = ['fid', 'iid', 'phen','sex','cohort','duo','family','is_eur','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10', 
        'PC1_PAT','PC2_PAT','PC3_PAT','PC4_PAT','PC5_PAT','PC6_PAT','PC7_PAT','PC8_PAT','PC9_PAT','PC10_PAT', 
        'PC1_MAT','PC2_MAT','PC3_MAT','PC4_MAT','PC5_MAT','PC6_MAT','PC7_MAT','PC8_MAT','PC9_MAT','PC10_MAT']
df_pca = df_pca[cols]

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_pca['iid'] = df_pca.apply(lambda row: rename_samples_dict[row['iid']] if row['iid'] in rename_samples_dict else row['iid'], axis=1)
#display(df_pca)

# first filter for the kids, then add parents to them
df_pca_flt = df_pca.loc[df_pca.iid.isin(df_trios.sample_id)].copy()
df_pca_flt['rel'] = 'C'
print('df_pca_flt:')
display(df_pca_flt)

df_parents = pd.DataFrame()
for index, row in df_pca_flt.iterrows():
    sample = row['iid']
    fid = row['fid']
    phen = row['phen']
    is_eur = row['is_eur']
    pc_mat = [row[f'PC{x}_MAT'] for x in range(1,11)]
    pc_pat = [row[f'PC{x}_PAT'] for x in range(1,11)]
    dad = sample_dad_dict[sample]
    mom = sample_mom_dict[sample]
    
    dict_mat = {'fid': [fid], 'iid': [mom], 'phen': ['.'], 'sex': ['Female'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_mat.update({f'PC{x}': [pc_mat[x-1]] for x in range(1,11)})
    df_mat = pd.DataFrame(dict_mat)

    dict_pat = {'fid': [fid], 'iid': [dad], 'phen': ['.'], 'sex': ['Male'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_pat.update({f'PC{x}': [pc_pat[x-1]] for x in range(1,11)})
    df_pat = pd.DataFrame(dict_pat)
    
    if df_parents.shape[0] == 0 or mom not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_mat])
    if df_parents.shape[0] == 0 or dad not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_pat])

df_parents['rel'] = 'P'
print('df_parents:')
display(df_parents)

df_pca_flt = pd.concat([df_pca_flt, df_parents])
print('df_pca_flt:')
display(df_pca_flt)

df_pca_flt = pd.merge(df_pca_flt, df_meta[['Sample_ID', 'Affected', 'Affected_ASD']], how='inner', left_on='iid', right_on='Sample_ID')
df_pca_flt = pd.merge(df_pca_flt, df_cov[['SAMPLE', 'MEAN_COVERAGE']], how='inner', left_on='iid', right_on='SAMPLE')

df_pca_flt['Phenotype'] = df_pca_flt.apply(lambda row: 1 if row.Affected == 'Yes' else 0, axis=1)
df_pca_flt['Phenotype_ASD'] = df_pca_flt.apply(lambda row: 1 if row.Affected_ASD == 'Yes' else 0, axis=1)
df_pca_flt['Platform'] = df_pca_flt.apply(lambda row: get_plat(row.iid), axis=1)
#df_pca_flt.to_csv('investigate2.tsv', sep='\t', index=None)

df_pca_flt['SNV_CAR'] = df_pca_flt.apply(lambda row: 1 if row['iid'] in this_df_dn_inh.samples_all.tolist() else 0, axis=1)
df_pca_flt['SNV_GENE'] = df_pca_flt.apply(lambda row: ','.join(this_df_dn_inh.loc[this_df_dn_inh.samples_all == row['iid']].gene.tolist()), axis=1)

print(f"# samples with SNV vars: {df_pca_flt['SNV_CAR'].sum()}")
display(df_pca_flt)

print(df_pca_flt.sex.value_counts())
print(df_pca_flt.Phenotype.value_counts())
print(df_pca_flt.Phenotype_ASD.value_counts())
print(df_pca_flt.Affected.value_counts())
print(df_pca_flt.Affected_ASD.value_counts())
print(df_pca_flt.Platform.value_counts())
print('case status of kids:')
print(df_pca_flt.loc[df_pca_flt.phen!='.'].Affected.value_counts())
print('case status of parents:')
print(df_pca_flt.loc[df_pca_flt.phen=='.'].Affected.value_counts())
print('case status of kids:')
print(df_pca_flt.loc[df_pca_flt.phen!='.'].Affected_ASD.value_counts())
print('case status of parents:')
print(df_pca_flt.loc[df_pca_flt.phen=='.'].Affected_ASD.value_counts())
print('Number of families:')
print(df_pca_flt.fid.value_counts().shape)
print('Parents cases:')
print(df_pca_flt.loc[(df_pca_flt.rel=='P')&(df_pca_flt.Affected=='Yes')].SNV_CAR.value_counts())
print('Parents controls:')
print(df_pca_flt.loc[(df_pca_flt.rel=='P')&(df_pca_flt.Affected=='No')].SNV_CAR.value_counts())
print('Children cases:')
print(df_pca_flt.loc[(df_pca_flt.rel=='C')&(df_pca_flt.Affected=='Yes')].SNV_CAR.value_counts())
print('Children controls:')
print(df_pca_flt.loc[(df_pca_flt.rel=='C')&(df_pca_flt.Affected=='No')].SNV_CAR.value_counts())

print(df_pca_flt.Platform.value_counts())
print(df_pca_flt.loc[df_pca_flt.Platform=='PB'].fid.value_counts().shape)
print(df_pca_flt.loc[df_pca_flt.Platform=='ONT'].fid.value_counts().shape)

df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC2_MAT,PC3_MAT,PC4_MAT,PC5_MAT,PC6_MAT,PC7_MAT,PC8_MAT,PC9_MAT,PC10_MAT,rel
136,F0001,REACH000001,ASD,Male,1,trio,1,1,0.008627,-0.005607,...,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,C
137,F0026,REACH000026,ASD,Male,1,trio,1,0,0.095364,-0.045230,...,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,C
139,F0058,REACH000058,ASD,Male,1,trio,1,1,0.002422,-0.009216,...,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,C
140,F0065,REACH000065,ASD,Male,1,trio,1,0,0.252774,0.107933,...,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,C
146,F0078,REACH000086,ASD,Male,1,trio,2,0,0.044785,0.038065,...,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,F0290,REACH000762,ASD,Male,1,trio,1,1,-0.027901,0.000946,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
409,F0290,REACH000765,CON,Female,1,trio,1,1,-0.030809,-0.001867,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
410,F0290,REACH000766,CON,Female,1,trio,1,1,-0.028469,-0.005167,...,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,C
411,F0291,REACH000767,ASD,Male,1,trio,1,0,0.087690,-0.012273,...,-0.016917,-0.021358,0.151522,-0.035297,0.008383,0.005679,0.020179,-0.022316,0.002755,C


df_parents:


,fid,iid,phen,sex,cohort,duo,is_eur,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,rel
0,F0001,REACH000107,.,Female,1,.,1,-0.022403,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,P
0,F0001,REACH000226,.,Male,1,.,1,0.024023,-0.013482,-0.003936,0.129597,0.030587,-0.007727,-0.002212,-0.022159,0.005150,-0.000824,P
0,F0026,REACH000269,.,Female,1,.,0,0.100551,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,P
0,F0026,REACH000270,.,Male,1,.,0,0.091671,-0.037847,-0.022134,0.163193,0.007864,-0.006841,-0.023716,0.040800,0.006666,-0.012916,P
0,F0058,REACH000439,.,Female,1,.,1,-0.025327,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F0288,REACH000756,.,Male,1,.,0,0.204041,0.127559,0.190286,0.013553,0.006706,-0.001266,-0.010150,0.006605,-0.006867,0.006560,P
0,F0290,REACH000763,.,Female,1,.,1,-0.030630,0.003052,-0.009075,-0.014136,-0.005499,-0.008555,0.015644,0.004869,0.011004,-0.019684,P
0,F0290,REACH000764,.,Male,1,.,1,-0.027327,-0.004878,-0.003881,0.000495,0.003727,-0.001647,-0.008831,0.016730,-0.005315,0.006958,P
0,F0291,REACH000768,.,Female,1,.,0,0.091442,-0.016917,-0.021358,0.151522,-0.035297,0.008383,0.005679,0.020179,-0.022316,0.002755,P


df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC2_MAT,PC3_MAT,PC4_MAT,PC5_MAT,PC6_MAT,PC7_MAT,PC8_MAT,PC9_MAT,PC10_MAT,rel
136,F0001,REACH000001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,...,0.000410,0.002845,-0.019312,-0.013769,-0.014384,0.007249,0.002096,0.005399,-0.014983,C
137,F0026,REACH000026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,...,-0.047780,-0.017435,0.153686,0.005292,-0.009487,-0.020773,0.034190,0.015895,-0.011925,C
139,F0058,REACH000058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,...,-0.003289,0.003757,-0.004828,0.007650,0.005633,0.032103,0.009510,0.020007,0.016383,C
140,F0065,REACH000065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,...,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,C
146,F0078,REACH000086,ASD,Male,1,trio,2.0,0,0.044785,0.038065,...,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,F0288,REACH000756,.,Male,1,.,NaN,0,0.204041,0.127559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000763,.,Female,1,.,NaN,1,-0.030630,0.003052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0290,REACH000764,.,Male,1,.,NaN,1,-0.027327,-0.004878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
0,F0291,REACH000768,.,Female,1,.,NaN,0,0.091442,-0.016917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P


# samples with SNV vars: 68


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,Sample_ID,Affected,Affected_ASD,SAMPLE,MEAN_COVERAGE,Phenotype,Phenotype_ASD,Platform,SNV_CAR,SNV_GENE
0,F0001,REACH000001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,...,REACH000001,Yes,Yes,REACH000001,10.15,1,1,ONT,0,
1,F0026,REACH000026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,...,REACH000026,Yes,Yes,REACH000026,3.22,1,1,PB,1,USF2
2,F0058,REACH000058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,...,REACH000058,Yes,Yes,REACH000058,6.19,1,1,PB,1,"EOMES,EOMES"
3,F0065,REACH000065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,...,REACH000065,Yes,Yes,REACH000065,1.22,1,1,PB,1,MYO9B
4,F0078,REACH000086,ASD,Male,1,trio,2.0,0,0.044785,0.038065,...,REACH000086,Yes,Yes,REACH000086,2.70,1,1,PB,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,F0288,REACH000756,.,Male,1,.,NaN,0,0.204041,0.127559,...,REACH000756,No,No,REACH000756,1.55,0,0,PB,0,
239,F0290,REACH000763,.,Female,1,.,NaN,1,-0.030630,0.003052,...,REACH000763,No,No,REACH000763,3.36,0,0,PB,0,
240,F0290,REACH000764,.,Male,1,.,NaN,1,-0.027327,-0.004878,...,REACH000764,No,No,REACH000764,1.20,0,0,PB,0,
241,F0291,REACH000768,.,Female,1,.,NaN,0,0.091442,-0.016917,...,REACH000768,No,No,REACH000768,0.10,0,0,PB,0,


sex
Male      137
Female    106
Name: count, dtype: int64
Phenotype
0    163
1     80
Name: count, dtype: int64
Phenotype_ASD
0    167
1     76
Name: count, dtype: int64
Affected
No     163
Yes     80
Name: count, dtype: int64
Affected_ASD
No     167
Yes     76
Name: count, dtype: int64
Platform
PB     142
ONT    101
Name: count, dtype: int64
case status of kids:
Affected
Yes    76
No     41
Name: count, dtype: int64
case status of parents:
Affected
No     122
Yes      4
Name: count, dtype: int64
case status of kids:
Affected_ASD
Yes    72
No     45
Name: count, dtype: int64
case status of parents:
Affected_ASD
No     122
Yes      4
Name: count, dtype: int64
Number of families:
(63,)
Parents cases:
SNV_CAR
0    4
Name: count, dtype: int64
Parents controls:
SNV_CAR
0    92
1    30
Name: count, dtype: int64
Children cases:
SNV_CAR
0    46
1    30
Name: count, dtype: int64
Children controls:
SNV_CAR
0    33
1     8
Name: count, dtype: int64
Platform
PB     142
ONT    101
Name: count, dtyp

In [14]:
fts = ['PLIp9_cds', 'FDR_ASD_cds', 'FDR_ASD_utr']
#fts = ['PLIp9_cds']

df_main = pd.DataFrame(df_pca_flt)
df_main.set_index('iid', inplace=True)

display(df_main)

for ft in fts:
    new_col = f'genes_{ft}'
    df_main[new_col] = [set() for x in range(df_main.shape[0])]
    print(ft)
    this_df = df.loc[df[f'X_{ft}'] == 1]
    #display(this_df['SYMBOL'])
    this_symbols = this_df['SYMBOL'].str.split(',')
    #display(this_symbols)
    
    zs_thr = 3
    supp_thr = 2
    samples_col = f'LZS{zs_thr}_SAMPLES_SUPP{supp_thr}_Q'
    this_samples = this_df[samples_col].str.split(',')
    #display(this_samples)
    for this_symbol, this_sample_tmp in zip(this_symbols, this_samples):
        # 'REACH000544:-0.52|3.24:-37|123:-37|7x123|8'
        this_sample = [x.split(':')[0] for x in this_sample_tmp if x != '']
        if len(this_sample) > 0:
            for sample in this_sample:
                if sample in df_main.index:
                    df_main.at[sample, new_col].update(this_symbol)
    #display(df_main[new_col])
    df_main[new_col] = [','.join([xx for xx in x if xx != '.']) for x in df_main[new_col].tolist()]

display(df_main)
file_out = dir_fig + 'investigate_genes.tsv'
print(f'writing {file_out}')
df_main.to_csv(file_out, sep='\t', header=True, index=True)

,fid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,PC3,...,Sample_ID,Affected,Affected_ASD,SAMPLE,MEAN_COVERAGE,Phenotype,Phenotype_ASD,Platform,SNV_CAR,SNV_GENE
iid,,,,,,,,,,,,,,,,,,,,,
REACH000001,F0001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,-0.003588,...,REACH000001,Yes,Yes,REACH000001,10.15,1,1,ONT,0,
REACH000026,F0026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,-0.019472,...,REACH000026,Yes,Yes,REACH000026,3.22,1,1,PB,1,USF2
REACH000058,F0058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,-0.007762,...,REACH000058,Yes,Yes,REACH000058,6.19,1,1,PB,1,"EOMES,EOMES"
REACH000065,F0065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,0.199378,...,REACH000065,Yes,Yes,REACH000065,1.22,1,1,PB,1,MYO9B
REACH000086,F0078,ASD,Male,1,trio,2.0,0,0.044785,0.038065,0.054606,...,REACH000086,Yes,Yes,REACH000086,2.70,1,1,PB,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000756,F0288,.,Male,1,.,NaN,0,0.204041,0.127559,0.190286,...,REACH000756,No,No,REACH000756,1.55,0,0,PB,0,
REACH000763,F0290,.,Female,1,.,NaN,1,-0.030630,0.003052,-0.009075,...,REACH000763,No,No,REACH000763,3.36,0,0,PB,0,
REACH000764,F0290,.,Male,1,.,NaN,1,-0.027327,-0.004878,-0.003881,...,REACH000764,No,No,REACH000764,1.20,0,0,PB,0,


PLIp9_cds
FDR_ASD_cds
FDR_ASD_utr


,fid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,PC3,...,SAMPLE,MEAN_COVERAGE,Phenotype,Phenotype_ASD,Platform,SNV_CAR,SNV_GENE,genes_PLIp9_cds,genes_FDR_ASD_cds,genes_FDR_ASD_utr
iid,,,,,,,,,,,,,,,,,,,,,
REACH000001,F0001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,-0.003588,...,REACH000001,10.15,1,1,ONT,0,,"MIR4647,TTBK1,ARID1B,SERPING1,PDE2A,GABRB3,MIR...","ARID1B,MIR4466,GABRB3",CSNK1E
REACH000026,F0026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,-0.019472,...,REACH000026,3.22,1,1,PB,1,USF2,,,
REACH000058,F0058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,-0.007762,...,REACH000058,6.19,1,1,PB,1,"EOMES,EOMES","TGFBR3L,CARD11,MAP2K7",,
REACH000065,F0065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,0.199378,...,REACH000065,1.22,1,1,PB,1,MYO9B,,,
REACH000086,F0078,ASD,Male,1,trio,2.0,0,0.044785,0.038065,0.054606,...,REACH000086,2.70,1,1,PB,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000756,F0288,.,Male,1,.,NaN,0,0.204041,0.127559,0.190286,...,REACH000756,1.55,0,0,PB,0,,,,
REACH000763,F0290,.,Female,1,.,NaN,1,-0.030630,0.003052,-0.009075,...,REACH000763,3.36,0,0,PB,0,,"TGFBR3L,TMEM184B,MAP2K7","KHK,CGREF1",
REACH000764,F0290,.,Male,1,.,NaN,1,-0.027327,-0.004878,-0.003881,...,REACH000764,1.20,0,0,PB,0,,,,


writing investigate_trs/investigate_genes.tsv
